# Exercise 4 (Preprocessing)

In [1]:
import pandas as pd
df_wind = pd.read_csv('../data/data_wind.csv')

In [2]:
df_wind.head()

,HORA (UTC),"VENTO, DIREï¿½ï¿½O HORARIA (gr) (ï¿½ (gr))","VENTO, VELOCIDADE HORARIA (m/s)",UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),TEMPERATURA Mï¿½XIMA NA HORA ANT. (AUT) (ï¿½C),TEMPERATURA Mï¿½NIMA NA HORA ANT. (AUT) (ï¿½C),"UMIDADE RELATIVA DO AR, HORARIA (%)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)","PRECIPITAï¿½ï¿½O TOTAL, HORï¿½RIO (mm)","VENTO, RAJADA MAXIMA (m/s)",PRESSï¿½O ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSï¿½O ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)
0,12:00,0.809017,1.8,69.0,60.0,22.6,20.7,61.0,888.2,0.0,3.8,888.2,887.7
1,13:00,0.965926,2.7,62.0,55.0,24.2,22.5,55.0,888.4,0.0,4.7,888.4,888.2
2,14:00,0.891007,2.0,56.0,50.0,25.5,24.3,51.0,888.1,0.0,4.9,888.4,888.1
3,15:00,0.848048,2.5,52.0,44.0,27.4,25.0,44.0,887.4,0.0,5.8,888.1,887.4
4,16:00,0.224951,2.4,50.0,43.0,27.1,25.5,46.0,886.5,0.0,5.8,887.4,886.5


In [3]:
df_wind.columns

Index(['HORA (UTC)', 'VENTO, DIREï¿½ï¿½O HORARIA (gr) (ï¿½ (gr))',
       'VENTO, VELOCIDADE HORARIA (m/s)',
       'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
       'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)',
       'TEMPERATURA Mï¿½XIMA NA HORA ANT. (AUT) (ï¿½C)',
       'TEMPERATURA Mï¿½NIMA NA HORA ANT. (AUT) (ï¿½C)',
       'UMIDADE RELATIVA DO AR, HORARIA (%)',
       'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
       'PRECIPITAï¿½ï¿½O TOTAL, HORï¿½RIO (mm)', 'VENTO, RAJADA MAXIMA (m/s)',
       'PRESSï¿½O ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)',
       'PRESSï¿½O ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)'],
      dtype='object')

## Rename columns

In [4]:
new_columns = [
    'Hour',
    'Wind direction (gr)',
    'Wind speed (m/s)',
    'Relative humidity max (%)',
    'Relative humidity min (%)',
    'Temperature min (℃)',
    'Temperature max (℃)',
    'Relative humidity (%)',
    'Atmospheric pressure at station level (mB)',
    'Precipitation (mm)',
    'Wind gust max (m/s)',
    'Atmospheric pressure max (mB)',
    'Atmospheric pressure min (mB)'
]

df_wind.rename(columns=dict(zip(df_wind.columns, new_columns)), inplace=True)
df_wind.columns

Index(['Hour', 'Wind direction (gr)', 'Wind speed (m/s)',
       'Relative humidity max (%)', 'Relative humidity min (%)',
       'Temperature min (℃)', 'Temperature max (℃)', 'Relative humidity (%)',
       'Atmospheric pressure at station level (mB)', 'Precipitation (mm)',
       'Wind gust max (m/s)', 'Atmospheric pressure max (mB)',
       'Atmospheric pressure min (mB)'],
      dtype='object')

In [5]:
total_added = 0

for _ in range(23):
    hours = df_wind['Hour']
    idx = 0
    nb_added = 0

    while idx < len(hours) - 1:
        cur_hour = int(hours[idx].split(':')[0])
        next_hour = int(hours[idx + 1].split(':')[0])
        if next_hour != (cur_hour + 1) % 24:
            if (cur_hour + 1) % 24 < 10:
                new_str = "0"+str((cur_hour + 1) % 24)+":00"
            else:
                new_str = str((cur_hour + 1) % 24)+":00"
            new_row = pd.DataFrame({'Hour': [new_str]})
            df_wind = pd.concat([df_wind.iloc[:idx + nb_added + 1], new_row, df_wind.iloc[idx + nb_added + 1:]], ignore_index=True)
            nb_added += 1
        idx += 1
    total_added += nb_added

print(f"Added {total_added} empty rows")

Added 2583 empty rows


The hour is given as a string. Considered we have data for almost ten years, we can change the format of the hour, starting with a given year, for instance the year 2000.

In [6]:
import datetime as dt

start_date = dt.datetime(2000, 1, 1, int(df_wind['Hour'][0][:2]))

df_wind['Date'] = [start_date + dt.timedelta(hours=i) for i in range(len(df_wind['Hour']))]

## International system units

In [7]:
df_wind['Hour'] = df_wind['Hour'].apply(lambda x: int(x.split(':')[0]))

df_wind['Relative humidity max'] = df_wind['Relative humidity max (%)'].apply(lambda x: x/100)
df_wind.drop(columns=['Relative humidity max (%)'], inplace=True)

df_wind['Relative humidity min'] = df_wind['Relative humidity min (%)'].apply(lambda x: x/100)
df_wind.drop(columns=['Relative humidity min (%)'], inplace=True)

df_wind['Temperature max (K)'] = df_wind['Temperature max (℃)'].apply(lambda x: x+273.15)
df_wind.drop(columns=['Temperature max (℃)'], inplace=True)

df_wind['Temperature min (K)'] = df_wind['Temperature min (℃)'].apply(lambda x: x+273.15)
df_wind.drop(columns=['Temperature min (℃)'], inplace=True)

df_wind['Relative humidity'] = df_wind['Relative humidity (%)'].apply(lambda x: x/100)
df_wind.drop(columns=['Relative humidity (%)'], inplace=True)

df_wind['Atmospheric pressure at station level (Pa)'] = df_wind['Atmospheric pressure at station level (mB)'].apply(lambda x: x*100)
df_wind.drop(columns=['Atmospheric pressure at station level (mB)'], inplace=True)

df_wind['Precipitation (m)'] = df_wind['Precipitation (mm)'].apply(lambda x: x/1000)
df_wind.drop(columns=['Precipitation (mm)'], inplace=True)

df_wind['Atmospheric pressure max (Pa)'] = df_wind['Atmospheric pressure max (mB)'].apply(lambda x: x*100)
df_wind.drop(columns=['Atmospheric pressure max (mB)'], inplace=True)

df_wind['Atmospheric pressure min (Pa)'] = df_wind['Atmospheric pressure min (mB)'].apply(lambda x: x*100)
df_wind.drop(columns=['Atmospheric pressure min (mB)'], inplace=True)

df_wind.columns.sort_values()

Index(['Atmospheric pressure at station level (Pa)',
       'Atmospheric pressure max (Pa)', 'Atmospheric pressure min (Pa)',
       'Date', 'Hour', 'Precipitation (m)', 'Relative humidity',
       'Relative humidity max', 'Relative humidity min', 'Temperature max (K)',
       'Temperature min (K)', 'Wind direction (gr)', 'Wind gust max (m/s)',
       'Wind speed (m/s)'],
      dtype='object')

In [8]:
df_wind.head()

,Hour,Wind direction (gr),Wind speed (m/s),Wind gust max (m/s),Date,Relative humidity max,Relative humidity min,Temperature max (K),Temperature min (K),Relative humidity,Atmospheric pressure at station level (Pa),Precipitation (m),Atmospheric pressure max (Pa),Atmospheric pressure min (Pa)
0,12,0.809017,1.8,3.8,2000-01-01 12:00:00,0.69,0.60,293.85,295.75,0.61,88820.0,0.0,88820.0,88770.0
1,13,0.965926,2.7,4.7,2000-01-01 13:00:00,0.62,0.55,295.65,297.35,0.55,88840.0,0.0,88840.0,88820.0
2,14,0.891007,2.0,4.9,2000-01-01 14:00:00,0.56,0.50,297.45,298.65,0.51,88810.0,0.0,88840.0,88810.0
3,15,0.848048,2.5,5.8,2000-01-01 15:00:00,0.52,0.44,298.15,300.55,0.44,88740.0,0.0,88810.0,88740.0
4,16,0.224951,2.4,5.8,2000-01-01 16:00:00,0.50,0.43,298.65,300.25,0.46,88650.0,0.0,88740.0,88650.0


## Imputation

In [9]:
import warnings
warnings.filterwarnings('ignore')

data_wind_imputed = df_wind.interpolate(method='linear', axis=0)    

## Delete multicolinearity with VIF

In [11]:
from utils import delete_multicollinearity

import warnings
warnings.filterwarnings('ignore')

df_VIF = data_wind_imputed.drop(columns=['Date', 'Hour'])

df_wind_preprocessed = delete_multicollinearity(df_VIF, 'Wind speed (m/s)', 10)
df_wind_preprocessed = pd.concat([df_wind_preprocessed, data_wind_imputed[['Wind speed (m/s)', 'Date', 'Hour']]], axis=1)

Dropped column Atmospheric pressure min (Pa) with VIF: 953187.2 (0s)
Dropped column Temperature min (K) with VIF: 427532.4 (0s)
Dropped column Atmospheric pressure max (Pa) with VIF: 238868.4 (0s)
Dropped column Atmospheric pressure at station level (Pa) with VIF: 9345.0 (0s)
Dropped column Relative humidity with VIF: 592.7 (0s)
Dropped column Relative humidity max with VIF: 309.6 (0s)
Dropped column Temperature max (K) with VIF: 24.6 (0s)


In [ ]:
# df_wind_preprocessed = data_wind_imputed.drop(columns=[
#     'Atmospheric pressure min (Pa)',
#     'Temperature min (K)',
#     'Atmospheric pressure max (Pa)'
# ])

In [13]:
print(df_wind_preprocessed.shape)
print(df_wind_preprocessed.columns.to_list())

(90276, 7)
['Wind direction (gr)', 'Wind gust max (m/s)', 'Relative humidity min', 'Precipitation (m)', 'Wind speed (m/s)', 'Date', 'Hour']


In [14]:
df_wind_preprocessed.to_csv('../data/data_wind_preprocessed1.csv', index=False)